This notebook demonstrates the class distribution of the utilized Fake/True News datasets

In [ ]:
import os
import glob
import pandas
import matplotlib.pyplot as plt
import numpy as np
from transformers import squad_convert_examples_to_features
from bnn import download_data, RANDOM_SEED
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import IncrementalPCA, PCA

In [ ]:
SAMPLE_SIZE = 3000
COMPONENTS = 2

Preprocess Data

In [ ]:
if os.path.exists('True.csv') and os.path.exists('Fake.csv'):
    files = glob.glob('*.csv')
else:
    files = download_data()

frames = {}
for file in files:
    frames[file] = pandas.read_csv(file, usecols=['text'])
    if file == 'True.csv':
        frames[file]['label'] = np.ones(frames[file].size)
    else:
        frames[file]['label'] = np.zeros(frames[file].size)

dataset = pandas.concat(frames.values())

PCA: Plot for TF-IDF and BoW vectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
analyzer = PCA(n_components=COMPONENTS)

In [ ]:
figure = plt.figure(figsize=(16, 12), dpi=100)
for i in range(6):
    # subset of the larger dataset to perform principle component analysis on
    # (dataset is too large to process all at once)
    subset = dataset.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED + i)

    # split data and labels into x and y
    x = subset['text'].values
    y = subset['label'].values

    # generate embeddings for TF-IDF
    print(f'Iteration: {i + 1}')
    tfidf_embeddings = tfidf_vectorizer.fit_transform(x).toarray()
    print('TF-IDF matrix shape:', tfidf_embeddings.shape)

    # principle component analysis of the tf-idf matrix/embeddings (keep 2 components and plot)
    tfidf_pca = analyzer.fit_transform(tfidf_embeddings)
    print('PCA matrix shape:', tfidf_pca.shape)

    # plot data for this iteration
    figure.add_subplot(2, 3, i + 1)
    plt.scatter(tfidf_pca[y == 0, 0], tfidf_pca[y == 0, 1], color='b', s=10, label='False')
    plt.scatter(tfidf_pca[y == 1, 0], tfidf_pca[y == 1, 1], color='r', s=10, label='True')
    plt.title(f'PCA of TF-IDF Vectors (n={SAMPLE_SIZE}, iteration={i + 1})')
    plt.legend()

plt.show()


In [ ]:
bow_vectorizer = CountVectorizer()

figure = plt.figure(figsize=(16, 12), dpi=100)
for i in range(6):
    # subset of the larger dataset to perform principle component analysis on
    # (dataset is too large to process all at once)
    subset = dataset.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED + i)

    # split data and labels into x and y
    x = subset['text'].values
    y = subset['label'].values

    # generate embeddings for BOW
    print(f'Iteration: {i + 1}')
    bow_embeddings = bow_vectorizer.fit_transform(x).toarray()
    print('BOW matrix shape:', bow_embeddings.shape)

    # principle component analysis of the bow matrix/embeddings (keep 2 components and plot)
    bow_pca = analyzer.fit_transform(bow_embeddings)
    print('PCA matrix shape:', bow_pca.shape)

    # plot data for this iteration
    figure.add_subplot(2, 3, i + 1)
    plt.scatter(bow_pca[y == 0, 0], bow_pca[y == 0, 1], color='b', s=10, label='False')
    plt.scatter(bow_pca[y == 1, 0], bow_pca[y == 1, 1], color='r', s=10, label='True')
    plt.title(f'PCA of BOW Vectors (n={SAMPLE_SIZE}, iteration={i + 1}')
    plt.legend()

plt.show()


Incremental PCA: Plot for TF-IDF and BoW vectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
analyzer = IncrementalPCA(n_components=COMPONENTS)

In [ ]:
figure = plt.figure(figsize=(16, 12), dpi=100)
for i in range(6):
    # subset of the larger dataset to perform principle component analysis on
    # (dataset is too large to process all at once)
    subset = dataset.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED + i)

    # split data and labels into x and y
    x = subset['text'].values
    y = subset['label'].values

    # generate embeddings for TF-IDF
    print(f'Iteration: {i + 1}')
    tfidf_embeddings = tfidf_vectorizer.fit_transform(x).toarray()
    print('TF-IDF matrix shape:', tfidf_embeddings.shape)

    # principle component analysis of the tf-idf matrix/embeddings (keep 2 components and plot)
    tfidf_pca = analyzer.fit_transform(tfidf_embeddings)
    print('Inc. PCA matrix shape:', tfidf_pca.shape)

    # plot data for this iteration
    figure.add_subplot(2, 3, i + 1)
    plt.scatter(tfidf_pca[y == 0, 0], tfidf_pca[y == 0, 1], color='b', s=10, label='False')
    plt.scatter(tfidf_pca[y == 1, 0], tfidf_pca[y == 1, 1], color='r', s=10, label='True')
    plt.title(f'Inc. PCA of TF-IDF Vectors (n={SAMPLE_SIZE}, iteration={i + 1})')
    plt.legend()

plt.show()

In [17]:
bow_vectorizer = CountVectorizer()

figure = plt.figure(figsize=(16, 12), dpi=100)
for i in range(6):
    # subset of the larger dataset to perform principle component analysis on
    # (dataset is too large to process all at once)
    subset = dataset.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED + i)

    # split data and labels into x and y
    x = subset['text'].values
    y = subset['label'].values

    # generate embeddings for BOW
    print(f'Iteration: {i + 1}')
    bow_embeddings = bow_vectorizer.fit_transform(x).toarray()
    print('BOW matrix shape:', bow_embeddings.shape)

    # principle component analysis of the bow matrix/embeddings (keep 2 components and plot)
    bow_pca = analyzer.fit_transform(bow_embeddings)
    print('Inc. PCA matrix shape:', bow_pca.shape)

    # plot data for this iteration
    figure.add_subplot(2, 3, i + 1)
    plt.scatter(bow_pca[y == 0, 0], bow_pca[y == 0, 1], color='b', s=10, label='False')
    plt.scatter(bow_pca[y == 1, 0], bow_pca[y == 1, 1], color='r', s=10, label='True')
    plt.title(f'Inc. PCA of BOW Vectors (n={SAMPLE_SIZE}, iteration={i + 1}')
    plt.legend()

plt.show()
